In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('../'))
from utils.osm_helper import *
import geopandas as gp
from osmread import Node, Way, Relation
import datetime
import pandas as pd

In [2]:
CHILD_PARENT_PATH = '../data/osm_child_direct_parent.csv'
CHILD_PARENT = pd.read_csv(CHILD_PARENT_PATH, index_col=0)
CHILD_PARENT

D:\Program Files\Anaconda2\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,child,parent,lv,child_oid,child_otype,parent_oid,parent_otype
0,0,root,1,30066952,Node,NaN,NaN
98555,98646,root,1,294867028,Way,NaN,NaN
98554,98645,root,1,294867027,Way,NaN,NaN
98553,98644,root,1,294867026,Way,NaN,NaN
98552,98643,root,1,294867025,Way,NaN,NaN
98551,98642,root,1,294867024,Way,NaN,NaN
98550,98641,root,1,294867023,Way,NaN,NaN
98549,98640,root,1,294867022,Way,NaN,NaN
98548,98639,root,1,294867021,Way,NaN,NaN
98547,98638,root,1,294867020,Way,NaN,NaN


In [16]:
CHILD_PARENT.columns= [u'node', u'parent', u'lv', u'child_oid', u'child_otype', u'parent_oid',
       u'parent_otype']

In [35]:
a = [5147841 ]
for x in a :
    print CHILD_PARENT[CHILD_PARENT.parent_oid==x]

         child  parent  lv   child_oid child_otype  parent_oid parent_otype
11          11  149024   2    49716662        Node     5147841     Relation
148389  149059  149024   2     5170966    Relation     5147841     Relation
148390  149060  149024   2     5170966    Relation     5147841     Relation
2965      2965  149020   2   262249795        Node     5147841     Relation
114        114  149020   2    49729677        Node     5147841     Relation
4273      4283  149020   2   638521162        Node     5147841     Relation
4683      4693  149018   2   642885416        Node     5147841     Relation
4719      4729  149025   2   646190840        Node     5147841     Relation
4607      4617  149024   2   641964881        Node     5147841     Relation
4606      4616  149024   2   641964879        Node     5147841     Relation
4200      4210  149020   2   634011123        Node     5147841     Relation
1194      1194  149025   2    49785838        Node     5147841     Relation
1331      13

In [77]:
CHILD_PARENT[CHILD_PARENT.child_oid==41964978]
oid = 41964978
otype= 'Way'
CHILD_PARENT[(CHILD_PARENT.child_oid==oid)& (CHILD_PARENT.child_otype==otype)]

,child,parent,lv,child_oid,child_otype,parent_oid,parent_otype
19772,19858,root,1,41964978,Way,NaN,NaN


In [92]:
def get_sub_tree(tree_df, childid, looping=0):
    mask_itself = (tree_df.child==childid)
    mask_children =  (tree_df.parent==str(childid))
    assert looping<100, 'looping too much childid:'+str( childid)
    
    itself = tree_df[mask_itself]
    oid = itself.child_oid.values[0]
    otype = itself.child_otype.values[0]
    subtree = set([(oid, otype)])
    
    children_oid = tree_df[mask_children].child_oid.values
    children_otype = tree_df[mask_children].child_otype.values
    children_oid_otype = zip(children_oid, children_otype)
    children_id = tree_df[mask_children].child.values
    subtree.update(children_oid_otype)
    if len(children_oid)==0:
        return subtree, itself.shape

    for cid in children_id:
        coid_subtree, coid_shape = get_sub_tree(tree_df, cid, looping+1)
        subtree.update(coid_subtree)
    return subtree, itself.shape
    

get_sub_tree(CHILD_PARENT, 19858)

({(26708232, 'Way'),
  (26708259, 'Way'),
  (41964978, 'Way'),
  (66646399, 'Way'),
  (358958010, 'Node')},
 (1, 7))

In [3]:
def get_sub_tree(tree_df, node_id, looping=0):
    mask_node = (tree_df.node==node_id)
    mask_children =  (tree_df.parent==str(node_id))
    assert looping<100, 'looping too much childid:'+str( node_id)

    node = tree_df[mask_node]
    oid = node.child_oid.values[0]
    otype = node.child_otype.values[0]
    subtree = {((oid, otype),)}

    children_oid = tree_df[mask_children].child_oid.values
    children_otype = tree_df[mask_children].child_otype.values
    children_oid_otype = zip(children_oid, children_otype)
    children_id = tree_df[mask_children].node.values
    subtree.update(children_oid_otype)
    if len(children_oid)==0:
        return subtree, node.shape

    for cid in children_id:
        c_subtree, c_tree_shape = get_sub_tree(tree_df, cid, looping+1)
        subtree.update(c_subtree)
    return subtree, node.shape

In [4]:
({((11,22),)})

1

In [8]:
nodes_have_children = CHILD_PARENT[
    (CHILD_PARENT.child.apply(str).isin(CHILD_PARENT.parent.value_counts().index.tolist()))
    & (CHILD_PARENT.parent=='root')
].child.tolist()
nodes_have_children.sort()
len(nodes_have_children)

5928

In [5]:
sub_trees_root = CHILD_PARENT[CHILD_PARENT.parent=='root'].child
sub_trees_root.shape

(132085L,)

In [13]:
print datetime.datetime.now()
sub_trees = []
shape_error = []
assert_error = []
for child_id in nodes_have_children:
    try:
        subtree, shape = get_sub_tree(CHILD_PARENT,child_id)
    except Exception as e:
        assert_error.append((child_id, e))
        continue
    if shape!=(1,7):
        shape_error.append((child_id, subtree, shape))
    sub_trees.append(subtree)
print datetime.datetime.now()


2016-11-12 19:43:08.117000
2016-11-12 19:43:08.168000


In [20]:
nodes_have_no_child = set(sub_trees_root)-set(nodes_have_children)
nodes_have_no_child = CHILD_PARENT[CHILD_PARENT.node.isin(nodes_have_no_child)]
one_node_subtrees = nodes_have_no_child.apply(lambda x: set([(int(x.child_oid), x.child_otype)]),axis=1).tolist()
sub_trees.extend(one_node_subtrees)
print len(sub_trees)


132085


In [140]:
with open('../data/subtree.txt') as f:
    set_read = eval(f.readlines()[0])

In [175]:
with open('../data/subtree.txt', 'w') as f:
    f.write(str(sub_trees))

In [105]:
OSM_DC_BBOX_PATH = '../data/dc_bbox.osm'
OSM_DC_BBOX = osm_container(OSM_DC_BBOX_PATH)
print OSM_DC_BBOX.data_size()

begin reading osm 2016-11-11 15:38:25.788000
finish reading osm 2016-11-11 15:39:43.812000
["len of <class 'osmread.elements.Relation'> = 3427", "len of <class 'osmread.elements.Way'> = 306332", "len of <class 'osmread.elements.Node'> = 2546260"]


In [176]:
print datetime.datetime.now()
otype_str_to_class = {'Node': Node, 'Way':Way, 'Relation':Relation}
from collections import defaultdict
all_tags = []
for subtree in sub_trees:
    tags = defaultdict(list)
    for oid, otype in subtree:
        obj = OSM_DC_BBOX.get_osm_obj_by_id(otype_str_to_class[otype], oid)
        for tag, value in obj.tags.items():
            tags[tag].append(value)
    all_tags.append(tags)
print datetime.datetime.now()

2016-11-11 18:56:00.313000
2016-11-11 18:56:02.921000


In [185]:
eval(str([dict(d) for d in all_tags[:10]]))

[{u'addr:city': [u'Washington'],
  u'addr:country': [u'US'],
  u'addr:housenumber': [u'738'],
  u'addr:postcode': [u'20003'],
  u'addr:state': [u'DC'],
  u'addr:street': [u'10th Street Southeast'],
  u'amenity': [u'school'],
  u'building': [u'yes'],
  u'created_by': [u'JOSM'],
  u'dcgis:captureyear': [u'19990331'],
  u'dcgis:dataset': [u'buildings'],
  u'dcgis:featurecode': [u'2000'],
  u'leisure': [u'pitch'],
  u'mappingdc:gid': [u'59154'],
  u'source': [u'dcgis'],
  u'sport': [u'baseball']},
 {u'name': [u'Shepherd Industrial Track'],
  u'old_railway_operator': [u'B&O'],
  u'railway': [u'level_crossing',
   u'level_crossing',
   u'disused',
   u'level_crossing',
   u'level_crossing',
   u'level_crossing',
   u'level_crossing',
   u'level_crossing',
   u'level_crossing'],
  u'service': [u'spur'],
  u'source': [u'USGS Ortho'],
  u'tiger:cfcc': [u'B11'],
  u'tiger:county': [u'District of Columbia, DC'],
  u'tiger:name_base': [u'Csx Railroad']},
 {u'bicycle': [u'no'],
  u'created_by': [u'

In [186]:
with open('../data/osm_tags_in_subtrees.txt','w') as f:
    f.write(str([dict(d) for d in all_tags]))